# A/B tests с Python




<a id="Libraries"></a>
##  Загрузка библиотек 

In [21]:
from typing import Union
from tqdm import tqdm
import plotly.express as px
import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

# Подготовка данных

In [22]:


data = pd.read_excel('/Users/anna/Desktop/АВ тесты/ДЗ семинар8/AB_Test_Results.xlsx')

data.head()

/Users/anna/Desktop/АВ тесты/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [24]:
data.shape

(10000, 3)

In [25]:
data.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


Посмотрим на группы отдельно

In [26]:
control = data[data['VARIANT_NAME'] == 'control'].copy(deep=True)
treatment = data[data['VARIANT_NAME'] == 'variant'].copy(deep=True)

In [27]:
control.describe()

,USER_ID,REVENUE
count,4984.000000,4984.000000
mean,4989.436798,0.129013
std,2905.145109,3.007524
min,2.000000,0.000000
25%,2466.000000,0.000000
50%,4964.500000,0.000000
75%,7576.250000,0.000000
max,10000.000000,196.010000


In [28]:
treatment.describe()

,USER_ID,REVENUE
count,5016.000000,5016.000000
mean,4972.776914,0.070070
std,2876.320625,1.314802
min,3.000000,0.000000
25%,2476.500000,0.000000
50%,4958.500000,0.000000
75%,7415.250000,0.000000
max,10000.000000,58.630000


Пока мы еще ничего не посчитали, но уже можно заметить, что средняя и максимальная выручка в первой группе больше, чем в тестовой. 
Все мы знаем, что среднее очень неустойчиво к выбросам, так что нам необходимо будет это учесть.

# Применение статистических критериев

In [29]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

## Метрика выручка на юзера

In [30]:

fig = px.histogram(data[data['REVENUE'] !=0],
                   x='REVENUE',
                   color = 'VARIANT_NAME',
                   title='REVENUE',
                   marginal = 'box',
                   nbins = 50,
                   barmode='overlay')
fig.show()

По гистограмме видим, что контрольная и тестовая группы близки по значениям выручки ,  и в обоих группах есть выбросы.

In [31]:
from tqdm import tqdm
continious_result(control, treatment, column='REVENUE')

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:02<00:00, 4890.54it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.025394,0.203305,0.755041,"[-0.018, 0.16]",0.058011


** Делаем вывод: ошибки первого и второго рода слишком велики, доверительный интервал включает 0, значит, нет статистически значимых отличий в контрольной и тестовой группе. Значит, не рекомендуем выкатывать изменение .